In [9]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
from sklearn.preprocessing import MinMaxScaler


In [10]:
df_features = pd.read_csv('features/features_interpolated.csv', encoding='latin-1', engine='python')
df_features.drop(columns=['Unnamed: 0'], inplace=True)
immigration = pd.read_csv('data/OECD_acquisition_data_interpolated.csv', encoding='latin-1', engine='python')
immigration.drop(columns=['Unnamed: 0'], inplace=True)

In [11]:
print(df_features.head())
print(immigration.head())

  Country  Year  Carbon Emissions  Education Expenditure   
0     AUS  2000          339446.6               4.887310  \
1     AUS  2001          345645.0               4.889552   
2     AUS  2002          353371.3               4.891794   
3     AUS  2003          352581.1               4.894036   
4     AUS  2004          365808.0               4.896278   

   Foreign Direct Investment (FDI) Inflows           GDP  Health Expenditure   
0                             1.489298e+10  4.158513e+11            7.599617  \
1                             1.071713e+10  3.793582e+11            7.682723   
2                             1.465632e+10  3.955808e+11            7.878076   
3                             8.985246e+09  4.674980e+11            7.882926   
4                             4.290767e+10  6.143264e+11            8.090034   

   Inflation Rate  Internet Penetration  Life Expectancy   
0        4.457435             46.756116        79.234146  \
1        4.407135             52.68926

For each year, create a graph based on the similarities of the countries

In [12]:
# get all features names except country and year
features = df_features.columns[2:]
years = df_features['Year'].unique()
countries = df_features['Country'].unique()

print(features)
print(years)
print(countries)

Index(['Carbon Emissions', 'Education Expenditure',
       'Foreign Direct Investment (FDI) Inflows', 'GDP', 'Health Expenditure',
       'Inflation Rate', 'Internet Penetration', 'Life Expectancy',
       'Renewable Energy Production', 'Unemployment Rate'],
      dtype='object')
[2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013
 2014 2015 2016 2017 2018 2019 2020 2021]
['AUS' 'AUT' 'BEL' 'CAN' 'CHE' 'CHL' 'DEU' 'DNK' 'ESP' 'FIN' 'FRA' 'GBR'
 'GRC' 'HUN' 'IRL' 'ISL' 'ITA' 'LUX' 'LVA' 'MEX' 'NLD' 'NOR' 'NZL' 'POL'
 'SVN' 'SWE' 'USA']


In [13]:
# create a threshold for each feature similarity score to be considered
threshold_ce = 0.5
threshold_ee = 0.5
threshold_fdi = 0.5
threshold_gpd = 0.5
threshold_he = 0.5
threshold_ir = 0.5
threshold_ip = 0.5
threshold_le = 0.5
threshold_rep = 0.5
threshold_ur = 0.5

# Set the minimum number of threshold crossings required
min_threshold_crossings = 5

similarity_thresholds = [threshold_ce, threshold_ee, threshold_fdi, threshold_gpd, threshold_he,
                         threshold_ir, threshold_ip, threshold_le, threshold_rep, threshold_ur]

In [14]:
def compute_similarities(df):
    # similarity_ce = cosine_similarity(df['Carbon Emissions'].values.reshape(-1, 1), df['Carbon Emissions'].values.reshape(-1, 1))
    # similarity_ee = cosine_similarity(df['Education Expenditure'].values.reshape(-1, 1), df['Education Expenditure'].values.reshape(-1, 1))
    # similarity_fdi = cosine_similarity(df['Foreign Direct Investment (FDI) Inflows'].values.reshape(-1, 1), df['Foreign Direct Investment (FDI) Inflows'].values.reshape(-1, 1))
    # similarity_gpd = cosine_similarity(df['GDP'].values.reshape(-1, 1), df['GDP'].values.reshape(-1, 1))
    # similarity_he = cosine_similarity(df['Health Expenditure'].values.reshape(-1, 1), df['Health Expenditure'].values.reshape(-1, 1))
    # similarity_ir = cosine_similarity(df['Inflation Rate'].values.reshape(-1, 1), df['Inflation Rate'].values.reshape(-1, 1))
    # similarity_ip = cosine_similarity(df['Internet Penetration'].values.reshape(-1, 1), df['Internet Penetration'].values.reshape(-1, 1))
    # similarity_le = cosine_similarity(df['Life Expectancy'].values.reshape(-1, 1), df['Life Expectancy'].values.reshape(-1, 1))
    # similarity_rep = cosine_similarity(df['Renewable Energy Production'].values.reshape(-1, 1), df['Renewable Energy Production'].values.reshape(-1, 1))
    # similarity_ur = cosine_similarity(df['Unemployment Rate'].values.reshape(-1, 1), df['Unemployment Rate'].values.reshape(-1, 1))
    
    similarity_ce = cdist(df['Carbon Emissions'].values.reshape(-1, 1), df['Carbon Emissions'].values.reshape(-1, 1), metric='canberra')
    similarity_ee = cdist(df['Education Expenditure'].values.reshape(-1, 1), df['Education Expenditure'].values.reshape(-1, 1), metric='canberra')
    similarity_fdi = cdist(df['Foreign Direct Investment (FDI) Inflows'].values.reshape(-1, 1), df['Foreign Direct Investment (FDI) Inflows'].values.reshape(-1, 1), metric='canberra')
    similarity_gpd = cdist(df['GDP'].values.reshape(-1, 1), df['GDP'].values.reshape(-1, 1), metric='canberra')
    similarity_he = cdist(df['Health Expenditure'].values.reshape(-1, 1), df['Health Expenditure'].values.reshape(-1, 1), metric='canberra')
    similarity_ir = cdist(df['Inflation Rate'].values.reshape(-1, 1), df['Inflation Rate'].values.reshape(-1, 1), metric='canberra')
    similarity_ip = cdist(df['Internet Penetration'].values.reshape(-1, 1), df['Internet Penetration'].values.reshape(-1, 1), metric='canberra')
    similarity_le = cdist(df['Life Expectancy'].values.reshape(-1, 1), df['Life Expectancy'].values.reshape(-1, 1), metric='canberra')
    similarity_rep = cdist(df['Renewable Energy Production'].values.reshape(-1, 1), df['Renewable Energy Production'].values.reshape(-1, 1), metric='canberra')
    similarity_ur = cdist(df['Unemployment Rate'].values.reshape(-1, 1), df['Unemployment Rate'].values.reshape(-1, 1), metric='canberra')
    
    return similarity_ce, similarity_ee, similarity_fdi, similarity_gpd, similarity_he, \
            similarity_ir, similarity_ip, similarity_le, similarity_rep, similarity_ur


In [15]:
graphs=[]

# for each year create the graph
for year in years:
    print(year)
    # empty graph
    graph = nx.Graph()
    
    df = df_features[df_features['Year'] == year]
    
    # Create a MinMaxScaler object
    scaler = MinMaxScaler()

    # Normalize the features before computing similarity
    normalized_features = scaler.fit_transform(df[['Carbon Emissions', 'Education Expenditure', 'Foreign Direct Investment (FDI) Inflows',
                                                'GDP', 'Health Expenditure', 'Inflation Rate', 'Internet Penetration',
                                                'Life Expectancy', 'Renewable Energy Production', 'Unemployment Rate']])

    # Create a new DataFrame with the normalized features
    df_normalized = pd.DataFrame(normalized_features, columns=['Carbon Emissions', 'Education Expenditure', 'Foreign Direct Investment (FDI) Inflows',
                                                            'GDP', 'Health Expenditure', 'Inflation Rate', 'Internet Penetration',
                                                            'Life Expectancy', 'Renewable Energy Production', 'Unemployment Rate'])

    # Compute similarity using the normalized features
    similarity_ce, similarity_ee, similarity_fdi, similarity_gpd, similarity_he, \
            similarity_ir, similarity_ip, similarity_le, similarity_rep, similarity_ur = compute_similarities(df_normalized)
            
    # Add countries as nodes to the graph and add all their features as node attributes
    for i, row in df.iterrows():
        graph.add_node(row['Country'], ce=row['Carbon Emissions'],
                    ee=row['Education Expenditure'], fdi=row['Foreign Direct Investment (FDI) Inflows'],
                    gpd=row['GDP'], he=row['Health Expenditure'],
                    ir=row['Inflation Rate'], ip=row['Internet Penetration'],
                    le=row['Life Expectancy'], rep=row['Renewable Energy Production'],
                    ur=row['Unemployment Rate'])

        
    # Add edges based on similarity scores and threshold crossings
    num_countries = len(countries)
    for i in range(num_countries):
        for j in range(i + 1, num_countries):
            country1 = countries[i]
            country2 = countries[j]
            # Check if the nodes already exist in the graph
            if country1 in graph.nodes() and country2 in graph.nodes():
                similarity_scores = [similarity_ce[i][j], similarity_ee[i][j], similarity_fdi[i][j], similarity_gpd[i][j], similarity_he[i][j],
                                     similarity_ir[i][j], similarity_ip[i][j], similarity_le[i][j], similarity_rep[i][j], similarity_ur[i][j]]

                # Count the number of threshold crossings
                threshold_crossings = sum(score > threshold for score, threshold in zip(similarity_scores, similarity_thresholds))

                if threshold_crossings >= min_threshold_crossings:
                    edge_value = immigration[(immigration['CO2'] == country1) & (immigration['COU'] == country2) & (immigration['Year'] == year)]['Value'].values[0]
                    graph.add_edge(country1, country2, weight=sum(similarity_scores), edge_value=edge_value)  # Add edge with maximum similarity score as weight and edge value
    
    # Calculate the number of edges and nodes in the graph
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()

    # Calculate the maximum number of edges possible in an undirected graph
    max_edges = (num_nodes * (num_nodes - 1)) / 2

    # Calculate the sparsity of the graph
    sparsity = num_edges / max_edges

    # Print the sparsity
    print("Graph Sparsity:", sparsity)
    
    graphs.append(graph)

2000


Graph Sparsity: 0.5612535612535613
2001
Graph Sparsity: 0.5071225071225072
2002
Graph Sparsity: 0.5356125356125356
2003
Graph Sparsity: 0.4843304843304843
2004
Graph Sparsity: 0.4444444444444444
2005
Graph Sparsity: 0.31339031339031337
2006
Graph Sparsity: 0.4444444444444444
2007
Graph Sparsity: 0.3418803418803419
2008
Graph Sparsity: 0.452991452991453
2009
Graph Sparsity: 0.31339031339031337
2010
Graph Sparsity: 0.3333333333333333
2011
Graph Sparsity: 0.31339031339031337
2012
Graph Sparsity: 0.3190883190883191
2013
Graph Sparsity: 0.31054131054131057
2014
Graph Sparsity: 0.3076923076923077
2015
Graph Sparsity: 0.396011396011396
2016
Graph Sparsity: 0.33618233618233617
2017
Graph Sparsity: 0.3418803418803419
2018
Graph Sparsity: 0.2792022792022792
2019
Graph Sparsity: 0.41025641025641024
2020
Graph Sparsity: 0.19373219373219372
2021


IndexError: index 0 is out of bounds for axis 0 with size 0